### cardiffnlp/twitter-roberta-base-sentiment
https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment?text=I+like+you.+I+love+you



### No. 2

In [1]:
!pip install datasets
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from transformers import RobertaTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import torch
from transformers import get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import os
from transformers import EarlyStoppingCallback

os.environ["TRANSFORMERS_VERBOSITY"] = "error"
os.environ["WANDB_DISABLED"] = "true"

# Load data
train_df = pd.read_csv('LT-EDI-ACL2022_Dataset/train.csv')
dev_df = pd.read_csv('LT-EDI-ACL2022_Dataset/dev.tsv', sep='\t')
test_df = pd.read_csv('LT-EDI-ACL2022_Dataset/test.csv')

# Drop unnecessary columns
train_df.drop("pid", axis=1, inplace=True)
dev_df.drop("PID", axis=1, inplace=True)
test_df.drop("pid", axis=1, inplace=True)

# Rename columns for dev_df
dev_df = dev_df.rename(columns={'Text data': 'text', 'Label': 'labels'})

# Map label names to integers
label_mapping = {'severe': 0, 'moderate': 1, 'not depression': 2}
dev_df['labels'] = dev_df['labels'].map(label_mapping)

# Convert DataFrame to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(dev_df)
test_dataset = Dataset.from_pandas(test_df)

# Merge datasets into a DatasetDict
dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

# Tokenizer
model_checkpoint = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = RobertaTokenizer.from_pretrained(model_checkpoint)

# Tokenize
def preprocess_function(examples):
    cleaned_texts = [str(text) if text else "" for text in examples["text"]]
    return tokenizer(cleaned_texts, truncation=True, max_length=128)

tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Load the model
model = RobertaForSequenceClassification.from_pretrained(model_checkpoint, num_labels=3, ignore_mismatched_sizes=True)  # 3 classes

# Data Collector
data_collator = DataCollatorWithPadding(tokenizer)

# Evaluation
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = torch.argmax(torch.tensor(predictions), dim=-1)
    accuracy = (preds == torch.tensor(labels)).float().mean().item()
    return {"accuracy": accuracy}

# parameters
training_args = TrainingArguments(
    output_dir="./results_2",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs_2',
    logging_steps=10,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

# optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)

# learning rate scheduler
num_training_steps = len(tokenized_datasets["train"]) // training_args.per_device_train_batch_size * training_args.num_train_epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=num_training_steps)

early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=2)

# Trainer class
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    optimizers=(optimizer, scheduler),
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback]
)

# Train
trainer.train()

# Evaluate the model
print("Validation metrics:")
val_metrics = trainer.evaluate(eval_dataset=tokenized_datasets["validation"])
print(val_metrics)

print("Test metrics:")
test_metrics = trainer.evaluate(eval_dataset=tokenized_datasets["test"])
print(test_metrics)

# Save
model.save_pretrained('./trained_model_2')
tokenizer.save_pretrained('./trained_model_2')

# Get predictions for the test set using the trained model
def get_predictions_and_logits(trainer, dataset):
    predictions, labels, metrics = trainer.predict(dataset)

    # Convert logits to predicted labels (0 or 1)
    predicted_labels = np.argmax(predictions, axis=-1)

    return predictions, predicted_labels, labels

# Get predictions, logits, and true labels for the test dataset
predictions, predicted_labels, true_labels = get_predictions_and_logits(trainer, tokenized_datasets["test"])

# Convert logits to pandas DataFrame for easy manipulation
logits_df = pd.DataFrame(predictions, columns=["logit_" + str(i) for i in range(predictions.shape[1])])

# Create a DataFrame with text, true labels, and predicted labels
results_df = pd.DataFrame({
    'text': tokenized_datasets["test"]["text"],
    'label': true_labels,
    'label_pred': predicted_labels
})

# Concatenate logits DataFrame with the results_df
final_df = pd.concat([results_df, logits_df], axis=1)
final_df.to_csv("test_predictions_with_logits_2.csv", index=False)

# Filter out misclassified samples
misclassified_df = results_df[results_df['label'] != results_df['label_pred']]
misclassified_df.to_csv('misclassified_samples_2.csv', index=False)

# Count per label
misclassified_counts = misclassified_df['label'].value_counts()

# Display distribution
print(f"Misclassification distribution:\n{misclassified_counts}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-2-1d6d7ce227ea>:96: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.864000,0.717974,0.660810
2,0.701500,0.594613,0.741326
3,0.512200,0.426535,0.846753
4,0.396300,0.465886,0.864101
5,0.227800,0.504422,0.890347
6,0.109900,0.709927,0.888790
7,0.071000,0.774813,0.896130
8,0.039700,0.855603,0.894795
9,0.019500,0.856804,0.897242
10,0.035600,0.856166,0.898132


Validation metrics:


{'eval_loss': 0.8561655282974243, 'eval_accuracy': 0.8981316685676575, 'eval_runtime': 31.1401, 'eval_samples_per_second': 144.38, 'eval_steps_per_second': 9.024, 'epoch': 10.0}
Test metrics:
{'eval_loss': 3.123176336288452, 'eval_accuracy': 0.617257297039032, 'eval_runtime': 22.3918, 'eval_samples_per_second': 144.919, 'eval_steps_per_second': 9.066, 'epoch': 10.0}
Misclassification distribution:
label
1    743
2    387
0    112
Name: count, dtype: int64


### No. 4

In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from transformers import RobertaTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import torch
from transformers import get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import os
from transformers import EarlyStoppingCallback

In [3]:
# Load data
train_df = pd.read_csv('Preprocessed_Dataset/train.csv')
val_df = pd.read_csv('Preprocessed_Dataset/dev.csv')
test_df = pd.read_csv('Preprocessed_Dataset/test.csv')

# Convert DataFrame to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Merge datasets
dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

# Tokenizer: Load from last training
model_checkpoint = './trained_model_2'
tokenizer = RobertaTokenizer.from_pretrained(model_checkpoint)

# Tokenize
def preprocess_function(examples):
    cleaned_texts = [str(text) if text else "" for text in examples["text"]]
    return tokenizer(cleaned_texts, truncation=True, max_length=128)

tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Load the model
model = RobertaForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2, ignore_mismatched_sizes=True)  # 2 classes

# Data Collector
data_collator = DataCollatorWithPadding(tokenizer)

# Evaluation
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = torch.argmax(torch.tensor(predictions), dim=-1)
    accuracy = (preds == torch.tensor(labels)).float().mean().item()
    return {"accuracy": accuracy}

# parameters
training_args = TrainingArguments(
    output_dir="./results_4",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs_4',
    logging_steps=10,
)

# optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)

# learning rate scheduler
num_training_steps = len(tokenized_datasets["train"]) // training_args.per_device_train_batch_size * training_args.num_train_epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=num_training_steps)

# Trainer class
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    optimizers=(optimizer, scheduler),
    compute_metrics=compute_metrics,
)

# Train
trainer.train()

# Evaluate the model
print("Validation metrics:")
val_metrics = trainer.evaluate(eval_dataset=tokenized_datasets["validation"])
print(val_metrics)

print("Test metrics:")
test_metrics = trainer.evaluate(eval_dataset=tokenized_datasets["test"])
print(test_metrics)

# Save
model.save_pretrained('./trained_model_4')
tokenizer.save_pretrained('./trained_model_4')

# Get predictions for the test set using the trained model
def get_predictions_and_logits(trainer, dataset):
    predictions, labels, metrics = trainer.predict(dataset)

    # Convert logits to predicted labels (0 or 1)
    predicted_labels = np.argmax(predictions, axis=-1)

    return predictions, predicted_labels, labels

# Get predictions, logits, and true labels for the test dataset
predictions, predicted_labels, true_labels = get_predictions_and_logits(trainer, tokenized_datasets["test"])

# Convert logits to pandas DataFrame for easy manipulation
logits_df = pd.DataFrame(predictions, columns=["logit_" + str(i) for i in range(predictions.shape[1])])

# Create a DataFrame with text, true labels, and predicted labels
results_df = pd.DataFrame({
    'text': tokenized_datasets["test"]["text"],
    'label': true_labels,
    'label_pred': predicted_labels
})

# Concatenate logits DataFrame with the results_df
final_df = pd.concat([results_df, logits_df], axis=1)
final_df.to_csv("test_predictions_with_logits_4.csv", index=False)

# Filter out misclassified samples
misclassified_df = results_df[results_df['label'] != results_df['label_pred']]
misclassified_df.to_csv('misclassified_samples_4.csv', index=False)

# Count per label
misclassified_counts = misclassified_df['label'].value_counts()

# Display distribution
print(f"Misclassification distribution:\n{misclassified_counts}")

Map:   0%|          | 0/6184 [00:00<?, ? examples/s]

Map:   0%|          | 0/773 [00:00<?, ? examples/s]

Map:   0%|          | 0/774 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ./trained_model_2 and are newly initialized because the shapes did not match:
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-3-33ced45c6dda>:65:

Epoch,Training Loss,Validation Loss,Accuracy
1,0.119200,0.120020,0.971539
2,0.020400,0.134988,0.974127
3,0.081100,0.126718,0.979301
4,0.038700,0.162055,0.974127
5,0.000200,0.153494,0.976714
6,0.000100,0.160892,0.978008
7,0.000200,0.151779,0.976714
8,0.000100,0.162138,0.979301
9,0.000000,0.182434,0.976714
10,0.000000,0.179001,0.975420


Validation metrics:


{'eval_loss': 0.17900098860263824, 'eval_accuracy': 0.9754204154014587, 'eval_runtime': 4.544, 'eval_samples_per_second': 170.116, 'eval_steps_per_second': 10.784, 'epoch': 10.0}
Test metrics:
{'eval_loss': 0.1732030063867569, 'eval_accuracy': 0.9780361652374268, 'eval_runtime': 4.6212, 'eval_samples_per_second': 167.488, 'eval_steps_per_second': 10.603, 'epoch': 10.0}
Misclassification distribution:
label
1    11
0     6
Name: count, dtype: int64
